In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_02_fully_connected")' FastaiNotebook_02_fully_connected

Installing packages:
	.package(path: "/home/ubuntu/git/fastai_docs/dev_swift/FastaiNotebook_02_fully_connected")
		FastaiNotebook_02_fully_connected
With SwiftPM flags: []
Working in: /tmp/tmpcu6fd8ir/swift-install
/home/ubuntu/swift/usr/bin/swift-build: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/lib/swift/linux/libFoundation.so)
/home/ubuntu/swift/usr/bin/swift-build: /home/ubuntu/anaconda3/lib/libcurl.so.4: no version information available (required by /home/ubuntu/swift/usr/lib/swift/linux/libFoundation.so)
/home/ubuntu/swift/usr/bin/swiftc: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swiftc)
Compile Swift Module 'FastaiNotebook_02_fully_connected' (4 sources)
/home/ubuntu/swift/usr/bin/swiftc: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swiftc)

/home/ubuntu/swift/usr/bin/swift:

In [2]:
//export
import FastaiNotebook_02_fully_connected

import Foundation
import TensorFlow
import Path

## Does nn.Conv2d init work well?

In [ ]:
var (xTrain, yTrain, xValid, yValid) = loadMNIST(path: Path.home/".fastai"/"data"/"mnist_tst")
let (trainMean, trainStd) = (xTrain.mean(), xTrain.standardDeviation())
xTrain = normalize(xTrain, mean: trainMean, std: trainStd)
xValid = normalize(xValid, mean: trainMean, std: trainStd)

In [ ]:
xTrain = xTrain.reshaped(to: [xTrain.shape[0], 28, 28, 1])
xValid = xValid.reshaped(to: [xValid.shape[0], 28, 28, 1])
print(xTrain.shape, xValid.shape)

In [ ]:
let images = xTrain.shape[0]
let classes = xValid.max() + 1
let channels = 32

In [ ]:
var layer1 = FAConv2D<Float>(filterShape: (5, 5, 1, channels)) //Conv2D(1, nh, 5)

In [ ]:
let x = xValid[0..<100]

In [ ]:
x.shape

In [ ]:
extension Tensor where Scalar: TensorFlowFloatingPoint {
    func stats() -> (mean: Tensor, std: Tensor) {
        return (mean: self.mean(), std: self.standardDeviation())
    }
}

In [ ]:
(filter: layer1.filter.stats(), bias: layer1.bias.stats())

In [ ]:
let result = layer1(x)

In [ ]:
result.stats()

In [ ]:
// export
extension Tensor where Scalar: TensorFlowFloatingPoint {
    init(kaimingNormal shape: TensorShape, negativeSlope: Double = 1.0) {
        // Assumes Leaky ReLU nonlinearity
        let gain = Scalar(sqrt(2.0 / (1.0 + pow(negativeSlope, 2))))
        let spatialDimCount = shape.count - 2
        let receptiveField = shape[0..<spatialDimCount].contiguousSize
        let fanIn = shape[shape.count - 2] * receptiveField
        self.init(
            randomNormal: shape,
            stddev: gain / sqrt(Scalar(fanIn)),
            generator: &PhiloxRandomNumberGenerator.global
        )
    }
}

In [ ]:
layer1.filter = Tensor(kaimingNormal: layer1.filter.shape, negativeSlope: 1.0)
layer1(x).stats()

In [ ]:
// export
func leakyRelu<T: TensorFlowFloatingPoint>(
    _ x: Tensor<T>,
    negativeSlope: Double = 0.0
) -> Tensor<T> {
    return max(0, x) + T(negativeSlope) * min(0, x)
}

In [ ]:
layer1.filter = Tensor(kaimingNormal: layer1.filter.shape, negativeSlope: 0.0)
leakyRelu(layer1(x)).stats()

In [ ]:
var layer1 = FAConv2D<Float>(filterShape: (5, 5, 1, channels)) //Conv2D(1, nh, 5)
leakyRelu(layer1(x)).stats()

In [ ]:
layer1.filter.shape

In [ ]:
let spatialDimCount = layer1.filter.rank - 2
let receptiveField = layer1.filter.shape[0..<spatialDimCount].contiguousSize
receptiveField

In [ ]:
let filtersIn = layer1.filter.shape[2]
let filtersOut = layer1.filter.shape[3]
print(filtersIn, filtersOut)

In [ ]:
let fanIn = filtersIn * receptiveField
let fanOut = filtersOut * receptiveField
print(fanIn, fanOut)

In [ ]:
func gain(_ negativeSlope: Double) -> Double {
    return sqrt(2.0 / (1.0 + pow(negativeSlope, 2.0)))
}

In [ ]:
(gain(1.0), gain(0.0), gain(0.01), gain(0.1), gain(sqrt(5.0)))

In [ ]:
(2 * Tensor<Float>(randomUniform: [10000]) - 1).standardDeviation()

In [ ]:
1.0 / sqrt(3.0)

In [ ]:
//export
extension Tensor where Scalar: TensorFlowFloatingPoint {
    init(kaimingUniform shape: TensorShape, negativeSlope: Double = 1.0) {
        // Assumes Leaky ReLU nonlinearity
        let gain = Scalar(sqrt(2.0 / (1.0 + pow(negativeSlope, 2))))
        let spatialDimCount = shape.count - 2
        let receptiveField = shape[0..<spatialDimCount].contiguousSize
        let fanIn = shape[shape.count - 2] * receptiveField
        let bound = sqrt(Scalar(3.0)) * gain / sqrt(Scalar(fanIn))
        self = bound * (2 * Tensor(
            randomUniform: shape,
            generator: &PhiloxRandomNumberGenerator.global
        ) - 1)
    }
}

In [ ]:
layer1.filter = Tensor(kaimingUniform: layer1.filter.shape, negativeSlope: 0.0)
leakyRelu(layer1(x)).stats()

In [ ]:
layer1.filter = Tensor(kaimingUniform: layer1.filter.shape, negativeSlope: sqrt(5.0))
leakyRelu(layer1(x)).stats()

In [ ]:
public struct Model: Layer {
    public var conv1 = FAConv2D<Float>(
        filterShape: (5, 5, 1, 8),   strides: (2, 2), padding: .same, activation: relu
    )
    public var conv2 = FAConv2D<Float>(
        filterShape: (3, 3, 8, 16),  strides: (2, 2), padding: .same, activation: relu
    )
    public var conv3 = FAConv2D<Float>(
        filterShape: (3, 3, 16, 32), strides: (2, 2), padding: .same, activation: relu
    )
    public var conv4 = FAConv2D<Float>(
        filterShape: (3, 3, 32, 1),  strides: (2, 2), padding: .valid
    )
    public var flatten = Flatten<Float>()
    @differentiable
    public func call(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: conv1, conv2, conv3, conv4, flatten)
    }
}

In [ ]:
let y = Tensor<Float>(yValid[0..<100])
var model = Model()

In [ ]:
let prediction = model(x)
prediction.stats()

In [ ]:
let gradients = gradient(at: model) { model in
    meanSquaredError(predicted: model(x), expected: y)
}

gradients.conv1.filter.stats()

In [ ]:
for keyPath in [\Model.conv1, \Model.conv2, \Model.conv3, \Model.conv4] {
    model[keyPath: keyPath].filter = Tensor(kaimingUniform: model[keyPath: keyPath].filter.shape)
}

In [ ]:
let prediction = model(x)
prediction.stats()

In [ ]:
let gradients = gradient(at: model) { model in
    meanSquaredError(predicted: model(x), expected: y)
}

gradients.conv1.filter.stats()

## Export

In [3]:
notebookToScript(fname: (Path.cwd / "02a_why_sqrt5.ipynb").string)